#Analiza i Bazy Danych

Maciej Wieloch, 303080

gr. 1a

Wykonaj zapytania za pomocą skryptów pythona:

1. Ile kategorii filmów mamy w wypożyczalni?
2. Wyświetl listę kategorii w kolejności alfabetycznej.
3. Znajdź najstarszy i najmłodszy film do wypożyczenia.
4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?
5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?
6. Znajdź największą płatność wypożyczenia.
7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.
8. Gdzie mieszkają członkowie personelu?
9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?
10. Jakie kategorie filmów zostały wypożyczone przez klientów?
11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.
12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley


In [52]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')


1. Ile kategorii filmów mamy w wypożyczalni?

In [53]:
df = pd.read_sql('select category_id from category order by category_id desc limit 1',con=connection)
df

,category_id
0,16


2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [54]:
df = pd.read_sql('select name from category order by name asc',con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

a) najstarszy

In [55]:
df = pd.read_sql('select title, release_year, length from film order by release_year desc limit 1',con=connection)
df

,title,release_year,length
0,Chamber Italian,2006,117


b) najmłodszy

In [56]:
df = pd.read_sql('select title, release_year, length from film order by release_year asc limit 1',con=connection)
df

,title,release_year,length
0,Chamber Italian,2006,117


Wszystkie filmy są z tego samego roku zatem niezależnie od formuły, wyświetla się ten sam rekord.

4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [57]:
df = pd.read_sql("select count(*) from rental where rental_date between '2005-07-01 00:00:00'::timestamp and '2005-08-01 23:59:59'::timestamp",con=connection)
df

,count
0,7380


5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [58]:
df = pd.read_sql("select count(*) from rental where rental_date between '2010-01-01 00:00:00'::timestamp and '2011-02-01 23:59:59'::timestamp",con=connection)
df

,count
0,0


6. Znajdź największą płatność wypożyczenia.

In [59]:
df = pd.read_sql('select amount from payment order by amount desc limit 1',con=connection)
df

,amount
0,11.99


7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

a) Polska

In [60]:
df = pd.read_sql("select c.first_name, c.last_name, ci.city, a.district, co.country from customer c left join address a on c.address_id = a.address_id left join city ci on a.city_id = ci.city_id left join country co on ci.country_id = co.country_id where co.country='Poland'",con=connection)
df

,first_name,last_name,city,district,country
0,Brian,Wyman,Bydgoszcz,Kujawsko-Pomorskie,Poland
1,Sidney,Burleson,Czestochowa,Slaskie,Poland
2,Marjorie,Tucker,Jastrzebie-Zdrj,Slaskie,Poland
3,Leah,Curtis,Kalisz,Wielkopolskie,Poland
4,Ruben,Geary,Lublin,Lubelskie,Poland
5,Johnnie,Chisholm,Plock,Mazowieckie,Poland
6,Russell,Brinson,Tychy,Slaskie,Poland
7,Jimmie,Eggleston,Wroclaw,Dolnoslaskie,Poland


b) Nigeria

In [61]:
df = pd.read_sql("select c.first_name, c.last_name, ci.city, a.district, co.country from customer c left join address a on c.address_id = a.address_id left join city ci on a.city_id = ci.city_id left join country co on ci.country_id = co.country_id where co.country='Nigeria'",con=connection)
df

,first_name,last_name,city,district,country
0,Sonia,Gregory,Benin City,Edo & Delta,Nigeria
1,Rodney,Moeller,Deba Habe,Bauchi & Gombe,Nigeria
2,Velma,Lucas,Effon-Alaiye,Oyo & Osun,Nigeria
3,Marilyn,Ross,Ife,Oyo & Osun,Nigeria
4,Elsie,Kelley,Ikerre,Ondo & Ekiti,Nigeria
5,Gladys,Hamilton,Ilorin,Kwara & Kogi,Nigeria
6,Carol,Garcia,Kaduna,Kaduna,Nigeria
7,Olga,Jimenez,Ogbomosho,Oyo & Osun,Nigeria
8,Bertha,Ferguson,Ondo,Ondo & Ekiti,Nigeria
9,Tracey,Barrett,Owo,Ondo & Ekiti,Nigeria


c) Bangladesz

In [62]:
df = pd.read_sql("select c.first_name, c.last_name, ci.city, a.district, co.country from customer c left join address a on c.address_id = a.address_id left join city ci on a.city_id = ci.city_id left join country co on ci.country_id = co.country_id where co.country='Bangladesh'",con=connection)
df

,first_name,last_name,city,district,country
0,Stephen,Qualls,Dhaka,Dhaka,Bangladesh
1,Frank,Waggoner,Jamalpur,Dhaka,Bangladesh
2,Michelle,Clark,Tangail,Dhaka,Bangladesh


8. Gdzie mieszkają członkowie personelu?

In [63]:
df = pd.read_sql("select st.staff_id, st.first_name, st.last_name, city.city, c.country from staff st left join address a on st.address_id = a.address_id left join city on a.city_id = city.city_id left join country c on city.country_id = c.country_id",con=connection)
df

,staff_id,first_name,last_name,city,country
0,1,Mike,Hillyer,Lethbridge,Canada
1,2,Jon,Stephens,Woodridge,Australia


9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [64]:
df = pd.read_sql("select count(*) from staff st left join address a on st.address_id = a.address_id left join city on a.city_id = city.city_id left join country c on city.country_id = c.country_id where c.country='Argentina' OR c.country='Spain'",con=connection)
df

,count
0,0


10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [65]:
df = pd.read_sql("select distinct name from category cat left join film_category fcat on cat.category_id = fcat.category_id left join film f on fcat.film_id = f.film_id left join inventory inv on f.film_id = inv.film_id left join rental r on inv.inventory_id = r.inventory_id",con=connection)
df

,name
0,Horror
1,Comedy
2,Drama
3,Sci-Fi
4,Foreign
5,Classics
6,Games
7,Travel
8,New
9,Music


11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [66]:
df = pd.read_sql("select distinct name from category cat left join film_category fcat on cat.category_id = fcat.category_id left join film f on fcat.film_id = f.film_id left join inventory inv on f.film_id = inv.film_id left join rental r on inv.inventory_id = r.inventory_id left join customer cust on r.customer_id = cust.customer_id left join address addr on cust.address_id = addr.address_id left join city on addr.city_id = city.city_id left join country on city.country_id = country.country_id where country='United States'",con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [67]:
df = pd.read_sql("select actor.first_name, actor.last_name, film.title from film left join film_actor on film.film_id = film_actor.film_id left join actor on film_actor.actor_id = actor.actor_id where actor.first_name = 'Olympia' and actor.last_name='Pfeiffer' or actor.first_name = 'Julia' and actor.last_name = 'Zellweger' or actor.first_name = 'Ellen' and actor.last_name = 'Presley'",con=connection)
df

,first_name,last_name,title
0,Ellen,Presley,Bilko Anonymous
1,Ellen,Presley,Caribbean Liberty
2,Ellen,Presley,Casper Dragonfly
3,Ellen,Presley,Empire Malkovich
4,Ellen,Presley,Floats Garden
...,...,...,...
64,Julia,Zellweger,Outlaw Hanky
65,Julia,Zellweger,Panky Submarine
66,Julia,Zellweger,Rider Caddyshack
67,Julia,Zellweger,Won Dares
